In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../data/processed/application_train_clean.parquet")

In [4]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_YEARS,AGE_OUTLIER,INCOME_OUTLIER,CREDIT_OUTLIER
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,1.0,25.9,0,0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,45.9,0,0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,52.1,0,0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,52.0,0,0,0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,54.6,0,0,0


In [7]:
df = df.copy()
df.columns = df.columns.str.lower()

In [8]:
df.columns

Index(['sk_id_curr', 'target', 'name_contract_type', 'code_gender',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'amt_credit', 'amt_annuity',
       ...
       'amt_req_credit_bureau_hour', 'amt_req_credit_bureau_day',
       'amt_req_credit_bureau_week', 'amt_req_credit_bureau_mon',
       'amt_req_credit_bureau_qrt', 'amt_req_credit_bureau_year', 'age_years',
       'age_outlier', 'income_outlier', 'credit_outlier'],
      dtype='object', length=126)

In [9]:
df_eda = df[
    [
        "sk_id_curr",
        "target",
        "amt_income_total",
        "amt_credit",
        "name_education_type",
        "name_family_status",
        "cnt_children",
        "age_years",
        "days_employed",
        "flag_own_car",
    ]
].copy()

In [12]:
df_eda["employment_years"] = df_eda["days_employed"].apply(
    lambda x: -x / 365 if x < 0 else None
)

In [13]:
df_eda["employment_status"] = df_eda["days_employed"].apply(
    lambda x: "Not employed" if x == 365243 else "Employed"
)

In [14]:
def map_education(x):
    if x in ["Lower secondary"]:
        return "Low"
    elif x in ["Secondary / secondary special", "Incomplete higher"]:
        return "Medium"
    elif x in ["Higher education", "Academic degree"]:
        return "High"
    else:
        return "Other"

df_eda["education_group"] = df_eda["name_education_type"].apply(map_education)

In [15]:
def map_family(x):
    if x in ["Married", "Civil marriage"]:
        return "Married"
    elif x in ["Single / not married", "Separated"]:
        return "Single"
    elif x == "Widow":
        return "Other"
    else:
        return "Other"

df_eda["family_group"] = df_eda["name_family_status"].apply(map_family)

In [16]:
df_eda.head()

,sk_id_curr,target,amt_income_total,amt_credit,name_education_type,name_family_status,cnt_children,age_years,days_employed,flag_own_car,employment_years,employment_status,education_group,family_group
0,100002,1,202500.0,406597.5,Secondary / secondary special,Single / not married,0,25.9,-637,N,1.745205,Employed,Medium,Single
1,100003,0,270000.0,1293502.5,Higher education,Married,0,45.9,-1188,N,3.254795,Employed,High,Married
2,100004,0,67500.0,135000.0,Secondary / secondary special,Single / not married,0,52.1,-225,Y,0.616438,Employed,Medium,Single
3,100006,0,135000.0,312682.5,Secondary / secondary special,Civil marriage,0,52.0,-3039,N,8.326027,Employed,Medium,Married
4,100007,0,121500.0,513000.0,Secondary / secondary special,Single / not married,0,54.6,-3038,N,8.323288,Employed,Medium,Single


In [17]:
df_eda.to_csv(
    "../dashboard/customer_profile_eda.csv",
    index=False
)